# Eat Safe, Love

## Part 1: Set Up the Database (and Jupyter Notebook)
In terminal, import the data from establishments.json by running the following command: `mongoimport --db uk_food --collection establishments --file resources/establishments.json --jsonArray`

In [1]:
from pymongo import MongoClient
from pprint import pprint

In [2]:
mongo = MongoClient(port=27017)

In [3]:
mongo.list_database_names()

['admin', 'config', 'local', 'uk_food']

In [4]:
uk_food_db = mongo["uk_food"]

In [5]:
uk_food_db.list_collection_names()


['establishments']

In [6]:
pprint(uk_food_db["establishments"].find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 551803,
 'LocalAuthorityBusinessID': 'PI/000070948',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6781c368fcea3f5aa44aa56f'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/551803',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate':

In [7]:
uk_food_establishments = uk_food_db["establishments"]

## Part 2: Update the Database

### Part 2.1: Add the new data

In [8]:
if uk_food_establishments.find_one({"BusinessName":"Penang Flavours"}):

    print(f"Restaurant with BusinessName 'Penang Flavours' was already in database")

else:

    new_restaurant_data = {
        "BusinessName":"Penang Flavours",
        "BusinessType":"Restaurant/Cafe/Canteen",
        "BusinessTypeID":"",
        "AddressLine1":"Penang Flavours",
        "AddressLine2":"146A Plumstead Rd",
        "AddressLine3":"London",
        "AddressLine4":"",
        "PostCode":"SE18 7DY",
        "Phone":"",
        "LocalAuthorityCode":"511",
        "LocalAuthorityName":"Greenwich",
        "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
        "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
        "scores":{
            "Hygiene":"",
            "Structural":"",
            "ConfidenceInManagement":""
        },
        "SchemeType":"FHRS",
        "geocode":{
            "longitude":"0.08384000",
            "latitude":"51.49014200"
        },
        "RightToReply":"",
        "Distance":4623.9723280747176,
        "NewRatingPending":True
    }

    uk_food_establishments.insert_one(new_restaurant_data)
    pprint(uk_food_establishments.find_one({"BusinessName":"Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6781c378a27609b4cf1f53bf'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


### Part 2.2: Clean the new data

In [9]:
if uk_food_establishments.find_one({"BusinessName":"Penang Flavours"})["BusinessTypeID"] == 1:

    print("Restaurant with BusinessName 'Penang Flavours' already had BusinessTypeID of 1")

else:
    
    rcc_businesstypeid = uk_food_establishments.find_one(
        {"BusinessType":"Restaurant/Cafe/Canteen"},
        {"BusinessType":1, "BusinessTypeID":1}
    )
    
    uk_food_establishments.update_one(
        {"BusinessName":"Penang Flavours"},
        {"$set":{"BusinessTypeID":rcc_businesstypeid["BusinessTypeID"]}}
    )
    pprint(uk_food_establishments.find_one({"BusinessName":"Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6781c378a27609b4cf1f53bf'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


### Part 2.3: Remove establishments with LocalAuthorityName "Dover"

In [10]:
if uk_food_establishments.count_documents({"LocalAuthorityName":"Dover"}) == 0:

    print("Restaurants with LocalAuthorityName 'Dover' were already removed")

else:

    print("Documents before: " + str(uk_food_establishments.count_documents({"LocalAuthorityName":"Dover"})))
    uk_food_establishments.delete_many({"LocalAuthorityName":"Dover"})
    print("Documents after: " + str(uk_food_establishments.count_documents({"LocalAuthorityName":"Dover"})))


Documents before: 994
Documents after: 0


### Part 2.4: Convert latitude/longitude and RatingValue to numbers

In [11]:
uk_food_establishments.update_many(
    {"geocode.latitude":{"$type":"string"}, "geocode.longitude":{"$type":"string"}},
    [
        {"$set":{"geocode.latitude":{"$toDouble":"$geocode.latitude"}, "geocode.longitude":{"$toDouble":"$geocode.longitude"}}}
    ]
)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [12]:
none_ratingvalues = ["AwaitingInspection", "Exempt", "AwaitingPublication", "Awaiting Inspection", "Pass"]
uk_food_establishments.update_many(
    {"RatingValue":{"$in":none_ratingvalues}},
    [
        {"$set":{"RatingValue":None}}
    ]
)

uk_food_establishments.update_many(
    {"RatingValue":{"$type":"string"}},
    [
        {"$set":{"RatingValue":{"$toDouble":"$RatingValue"}}}
    ]
)

UpdateResult({'n': 34694, 'nModified': 34694, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [13]:
uk_food_establishments.find_one(
    {},
    {"RatingValue":1, "geocode":1}
)

{'_id': ObjectId('6781c368fcea3f5aa44aa855'),
 'RatingValue': 5.0,
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812}}